In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import mode
import glob
from itertools import combinations

# Ruta a los archivos CSV
ruta_archivos = "*.csv"  # Cambia esta ruta según tu carpeta
archivos = glob.glob(ruta_archivos)  # Obtiene una lista de archivos CSV en la carpeta

if not archivos:
    raise FileNotFoundError("No se encontraron archivos CSV en la ruta especificada.")

# Leer los datos y almacenarlos en una lista
datos = []
for archivo in archivos:
    data = pd.read_csv(archivo)
    if 'numero_de_cliente' not in data.columns or 'Predicted' not in data.columns:
        raise ValueError(f"El archivo {archivo} no contiene las columnas requeridas.")
    data = data[['numero_de_cliente', 'Predicted']].sort_values(by='numero_de_cliente').reset_index(drop=True)
    datos.append(data)

# Asegurarse de que todos tienen el mismo conjunto de clientes
for i in range(1, len(datos)):
    if not datos[0]['numero_de_cliente'].equals(datos[i]['numero_de_cliente']):
        raise ValueError("Los archivos no tienen los mismos clientes o el orden no coincide.")

# Generar combinaciones de archivos y aplicar majority voting
combinaciones = []
for r in range(2, len(archivos) + 1):  # Desde combinaciones de 2 hasta todos los archivos
    combinaciones.extend(combinations(range(len(archivos)), r))

# Procesar cada combinación
numero_de_cliente = datos[0]['numero_de_cliente']  # Mantener los IDs de los clientes
for combo in combinaciones:
    # Obtener los datos de los modelos seleccionados
    predicciones = [datos[i]['Predicted'].values for i in combo]
    predicciones = np.array(predicciones)  # Convertir a matriz numpy

    # Aplicar majority voting
    majority_votes, _ = mode(predicciones, axis=0)

    # Crear un DataFrame con los resultados
    resultados_df = pd.DataFrame({
        'numero_de_cliente': numero_de_cliente,
        'Predicted': majority_votes.flatten()
    })

    # Crear un nombre de archivo descriptivo para la combinación
    nombre_archivo = f"predicciones_comboo_{'_'.join(map(str, combo))}.csv"
    resultados_df.to_csv(nombre_archivo, index=False)

    print(f"Predicciones generadas para la combinación {combo} y guardadas en {nombre_archivo}.")
